In [23]:
from random import randint, seed
def shortestVector(matrix):
    """
    Returns the shortest vector of a given matrix.
    """
    minNormIndex = 0
    for i in range(1, matrix.nrows() - 1):
        if matrix[i].norm() < matrix[minNormIndex].norm():
            matrix[minNormIndex] = matrix[i]
    return matrix[minNormIndex]
    
    return sorted([(matrix[idx].norm(), idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, perimeter):
    """
    Creates random square matrix of given dimension with numbers from range (-perimeter, perimeter).
    """
    list = [randint(-perimeter, perimeter) for _ in range(dimension**2)]
    return matrix(ZZ, dimension, dimension, list)

def gram_matrix(matrix):
    """
    Calculate Gram matrix of given matrix.
    :param matrix: given matrix
    :return: Gram matrix of matrix
    """
    return matrix * matrix.transpose()


In [26]:
def comparsion (dimension, perimeter, B, B_red, v_min, x, G):
    """
    dimension : int
        Size of the square matrix B. Length of vectors x, y.
    perimeter : int
        Range for numbers of the matrix B: (- perimeter, perimeter).
    B : matrix 
        Square matrix of size dimension. Un-reduced.
    B_red : matrix
    v_min : vector (1 x dimension)
        Shortest vector of the B_red basis
    x : vector (1 x dimension)
        Vector giving the linear combination of the rows of the matrix B_red, which gives the shortest vector v_min.
    G : matrix
        Gramm matrix of B
    """
    for i in range(dimension):
        variables = [var('y_{}'.format(j)) for j in range(dimension)]
        y = deepcopy(variables)
        y[i] = x[i]
        # print("y[0]: ", y[0], type(y[0]))
        Y =  matrix(1, dimension, y)
        to_minimize = (Y * G * Y.transpose())[0]
        partials =  []
        eqs = []
        for variable in variables:
            derivative = diff(to_minimize, variable)[0]
            partials.append(derivative)
            eqs.append(derivative == 0)
        y.remove(y[i])
        tmp = list(map(list, solve(eqs, y, solution_dict=True)[0].items())) # jdu blejt 
        # https://www.geeksforgeeks.org/python-extract-dictionary-values-list-to-list/
        # now I am missing a single y_i I removed earlier, I want to add it for clarity later
        # absolutely horrendous code, please bear with me
        tmp.insert(i, [variables[i], x[i]])
        # horrendouc code -- end
        solutions = [i[1].n() for i in tmp]
        for j in range(dimension - 1):
            difference = x[j] - solutions[j]
            if difference > 1: 
                print(B)
                print(x, solutions, x - vector(solutions))
                print()
                return B
        print("Nope")
        return
    


comparsion(dimension, perimeter, B, B_red, v_min, x, G)

NameError: name 'perimeter' is not defined

In [27]:
def main():
    for i in range(100):
        seed(i) # sets random seed for randint
        dimension = 10
        perimeter = 500

        B = randomMatrix(dimension, perimeter)
        #print(ZZ.are_linearly_dependent(B.rows()))
        B_red = B.LLL()
        v_min = shortestVector(B_red)
        x = B.solve_left(v_min)
        G = gram_matrix(B)
        
        comparsion(dimension, perimeter, B, B_red, v_min, x, G)
print("done")

main()

done
Nope
Nope
Nope
[-257  106   57 -367 -122  437  118  -15  140   94]
[-433  120 -487  430  357  -20 -235   64 -261 -304]
[ 234  -19   53  356   62  -13  -94  154  381 -346]
[-263  150 -345  388  448   35 -101  259 -485  187]
[ 295 -435 -337  276  480  105 -457 -192  298 -469]
[ 343  386 -225  -16  109  236  442  399 -104  231]
[ 307  443  -63  -96  245  320   90  -45  487  458]
[-363  399 -126 -401 -464 -361    6 -278 -236  488]
[ 188  -54  297  141  375 -192  -69   19  353 -105]
[  87 -141   46   99  -83   98 -263  425 -156  198]
(-1, 3, -2, -3, -1, 0, 1, -1, 0, 1) [-1.00000000000000, 1.54322200740863, -1.45156902046662, -1.65905811977108, -0.685120211874281, -0.146100946326173, 0.772844896601338, -0.841757085081036, -0.0632746637741350, 0.604447563702363] (0.000000000000000, 1.45677799259137, -0.548430979533383, -1.34094188022892, -0.314879788125719, 0.146100946326173, 0.227155103398662, -0.158242914918964, 0.0632746637741350, 0.395552436297637)

[-259 -190 -395  238  -95  -10 -34

Nope
Nope
Nope
Nope
Nope
Nope
[ 380  125   27  388 -384 -174 -377  319 -184 -314]
[ 396  376  456 -279 -141 -378  -48  320  -33   36]
[ 446 -216  -44 -326 -180 -215   14  139 -497  205]
[  26 -455 -370  468  342 -267  -52 -341  476  111]
[ -70 -260 -225 -239  179  393  -53 -500   64 -293]
[ 380  179  448 -337   80 -442  253 -278  233  440]
[-159 -457 -480 -482  206  365  477  459 -392  -95]
[ 209   80    6  234  -95 -351  214 -326  113 -322]
[ 298  427  -60  -27  -26 -112  138 -396 -267 -310]
[ 437  442 -444  -51  357   10 -371  -17 -341 -307]
(-1, -6, 1, -6, -1, 3, -1, 9, -10, 5) [-1.00000000000000, -3.72968717623864, 0.881401063721849, -3.95788034760060, -0.541009229527843, 1.57597031021695, -0.887596223973282, 5.91897442533277, -6.51522317050085, 3.19049279877439] (0.000000000000000, -2.27031282376136, 0.118598936278151, -2.04211965239940, -0.458990770472157, 1.42402968978305, -0.112403776026718, 3.08102557466723, -3.48477682949915, 1.80950720122561)

Nope
Nope
[ 318 -242 -167 -298 